In [0]:
%python
df = spark.read.table("dev.sukeerthi_bronze.products")

In [0]:
%python
df.display()

In [0]:
%python
df_cleaned = df.dropDuplicates().dropna().drop("ingestion_date")

In [0]:
create or replace table dev.sukeerthi_silver.products_clean (product_id int, product_name string, product_category string, product_price double)

In [0]:
MERGE INTO dev.sukeerthi_silver.products_clean AS t
USING (
    SELECT * 
    FROM (
        SELECT 
            x.*, 
            ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY seqNum DESC) AS rnk 
        FROM dev.sukeerthi_bronze.products x
    ) AS ranked_products
    WHERE rnk = 1 AND operation != 'DELETE'
) AS s
ON t.product_id = s.product_id
WHEN MATCHED THEN 
    UPDATE SET 
        t.product_name = s.product_name, 
        t.product_category = s.product_category, 
        t.product_price = s.product_price
WHEN NOT MATCHED THEN 
    INSERT (product_id, product_name, product_category, product_price)
    VALUES (s.product_id, s.product_name, s.product_category, s.product_price)
WHEN NOT MATCHED BY SOURCE THEN 
    DELETE;

In [0]:
select * from dev.sukeerthi_silver.products_clean 

In [0]:
%python
df_cleaned.write.mode("overwrite").saveAsTable("dev.sukeerthi_silver.sales")

In [0]:
select * from dev.sukeerthi_bronze.sales

In [0]:
%python
df=spark.read.table("dev.sukeerthi_bronze.sales")

In [0]:
%python
df_cleaned=df.dropDuplicates().dropna().drop("ingestion_date")

In [0]:
%python
df_cleaned.write.mode("overwrite").saveAsTable("dev.sukeerthi_silver.sales_cleaned")